In [15]:
import SimpleITK as sitk
import numpy as np
from skimage.filters import threshold_otsu

def evaluate_mask_vs_image_edges(
    ct_path,
    pred_mask_path,
    sigma_mm=0.5,
    edge_threshold_method='otsu',  # or 'percentile'
    percentile=85,
    erosion_radius_vox=1,
    min_edge_component_vox=50
):
    """
    Compare a predicted segmentation (mask) to CT image edges.
    Returns mean, std, HD95, etc. in mm.
    """

    # --- Load CT and predicted mask
    ct = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_mask_test.nii")
    pred = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\mandible.nii.gz")
    pred = sitk.Cast(pred > 0, sitk.sitkUInt8)  # ensure binary

    spacing = ct.GetSpacing()
    mean_spacing = float(sum(spacing) / 3.0)
    sigma_pixels = sigma_mm / mean_spacing

    # --- Compute gradient magnitude (smoothed)
    grad = sitk.GradientMagnitudeRecursiveGaussian(ct, sigma_pixels)

    # --- Threshold gradient to create edge map
    grad_arr = sitk.GetArrayFromImage(grad)
    if edge_threshold_method == 'otsu':
        otsu_val = threshold_otsu(grad_arr[grad_arr > 0])
        edge_bin = sitk.Cast(grad > otsu_val, sitk.sitkUInt8)
    else:
        thresh_val = np.percentile(grad_arr[grad_arr > 0], percentile)
        edge_bin = sitk.Cast(grad > float(thresh_val), sitk.sitkUInt8)

    # --- Remove small connected components
    cc = sitk.ConnectedComponent(edge_bin)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(cc)
    keep_labels = [lab for lab in stats.GetLabels() if stats.GetNumberOfPixels(lab) >= min_edge_component_vox]

    filtered_edges = sitk.Image(edge_bin.GetSize(), sitk.sitkUInt8)
    filtered_edges.CopyInformation(edge_bin)
    for lab in keep_labels:
        filtered_edges = filtered_edges | sitk.Cast(cc == lab, sitk.sitkUInt8)

    # --- Restrict edges to a region near the segmentation (within 20 mm)
    dist_to_mask = sitk.SignedMaurerDistanceMap(pred, insideIsPositive=True, useImageSpacing=True)
    abs_dist = sitk.Abs(dist_to_mask)
    close_edges = filtered_edges & sitk.Cast(abs_dist < 20.0, sitk.sitkUInt8)

    edge_for_eval = close_edges if sitk.GetArrayFromImage(close_edges).sum() > 0 else filtered_edges

    # --- Get predicted boundary
   # --- Get predicted boundary
    eroded = sitk.BinaryErode(pred, [erosion_radius_vox]*3)
    boundary = pred - eroded
    boundary = sitk.Cast(boundary > 0, sitk.sitkUInt8)


    # --- Compute distance map (distance to nearest edge voxel)
    distance_map = sitk.SignedMaurerDistanceMap(edge_for_eval, insideIsPositive=True, useImageSpacing=True)
    distance_map = sitk.Abs(distance_map)

    # --- Sample distances at predicted boundary voxels
    b_arr = sitk.GetArrayFromImage(boundary)
    dist_arr = sitk.GetArrayFromImage(distance_map)
    indices = np.argwhere(b_arr > 0)
    if indices.size == 0:
        print("No boundary voxels found — check mask.")
        return None

    dvals = dist_arr[indices[:, 0], indices[:, 1], indices[:, 2]].astype(np.float64)

    # --- Compute metrics
    mean_d = float(np.mean(dvals))
    std_d = float(np.std(dvals))
    median_d = float(np.median(dvals))
    hd95_d = float(np.percentile(dvals, 95))
    pct1 = float(np.mean(dvals <= 1.0) * 100.0)
    pct2 = float(np.mean(dvals <= 2.0) * 100.0)
    pct3 = float(np.mean(dvals <= 3.0) * 100.0)

    results = {
        "mean_mm": mean_d,
        "std_mm": std_d,
        "median_mm": median_d,
        "hd95_mm": hd95_d,
        "pct_within_1mm": pct1,
        "pct_within_2mm": pct2,
        "pct_within_3mm": pct3,
        "n_boundary_voxels": int(indices.shape[0]),
    }

    return results


In [16]:
# Example single test
ct_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_mask_test.nii"
mask_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\mandible.nii.gz"

results = evaluate_mask_vs_image_edges(ct_path, mask_path)
print("Mandible:", results)


Mandible: {'mean_mm': 1.0018339184127882, 'std_mm': 2.4823100012138553, 'median_mm': 0.0, 'hd95_mm': 4.76837158203125, 'pct_within_1mm': 86.332784184514, 'pct_within_2mm': 90.47116968698518, 'pct_within_3mm': 90.47116968698518, 'n_boundary_voxels': 15175}


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, fixed
from skimage.filters import threshold_otsu

# --- Load your CT and segmentation first (change these paths)
ct_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_mask_test.nii"
mask_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\mandible.nii.gz"

ct = sitk.ReadImage(ct_path)
mask_img = sitk.ReadImage(mask_path)

ct_np = sitk.GetArrayFromImage(ct)
mask = sitk.GetArrayFromImage(mask_img).astype(bool)

# --- Compute gradient magnitude image (smoothed)
spacing = ct.GetSpacing()
sigma_mm = 1.0
mean_spacing = np.mean(spacing)
sigma_pixels = sigma_mm / mean_spacing
grad = sitk.GradientMagnitudeRecursiveGaussian(ct, sigma_pixels)
grad_np = sitk.GetArrayFromImage(grad)

# --- Compute Otsu threshold (for reference)
otsu_val = threshold_otsu(grad_np[grad_np > 0])

# --- Interactive viewer
def show_edges(slice_idx, thresh_val, method):
    plt.figure(figsize=(8, 8))

    # base image
    plt.imshow(ct_np[slice_idx, :, :], cmap='gray', vmin=-500, vmax=1000)

    # overlay segmentation
    plt.contour(mask[slice_idx, :, :], colors='r', linewidths=1, levels=[0.5])

    # compute thresholded edges for this slice
    if method == "Otsu":
        edge_mask = grad_np > otsu_val
    else:
        edge_mask = grad_np > thresh_val

    plt.contour(edge_mask[slice_idx, :, :], colors='cyan', linewidths=0.5, levels=[0.5])

    plt.title(f"Slice {slice_idx} — red: segmentation | cyan: edges ({method})")
    plt.axis('off')
    plt.show()

# --- Interactive controls
interact(
    show_edges,
    slice_idx=IntSlider(min=0, max=ct_np.shape[0]-1, step=1, value=ct_np.shape[0]//2),
    thresh_val=FloatSlider(min=0, max=np.percentile(grad_np, 99), step=0.1, value=otsu_val),
    method=Dropdown(options=["Otsu", "Manual"], value="Otsu", description="Threshold:")
);


interactive(children=(IntSlider(value=75, description='slice_idx', max=149), FloatSlider(value=0.1938398331403…

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from ipywidgets import interact, IntSlider, FloatSlider
from skimage.filters import threshold_otsu

# --- Load your CT and segmentation first (change these paths)
ct_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_WHOLE_CNS_cropped.nii"
mask_path = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\mandible.nii.gz"

ct = sitk.ReadImage(ct_path)
mask_img = sitk.ReadImage(mask_path)

ct_np_orig = sitk.GetArrayFromImage(ct)
mask = sitk.GetArrayFromImage(mask_img).astype(bool)

# --- Interactive viewer function
def show_edges(slice_idx, sigma_mm, hu_lower, threshold_percentile):
    # 1️⃣ Clamp CT for bone window
    ct_np = np.clip(ct_np_orig, hu_lower, 3000)
    
    # 2️⃣ Convert to SimpleITK image for Gaussian smoothing
    ct_sitk = sitk.GetImageFromArray(ct_np)
    spacing = ct.GetSpacing()
    mean_spacing = np.mean(spacing)
    sigma_pix = sigma_mm / mean_spacing

    # 3️⃣ Smoothed gradient magnitude
    grad = sitk.GradientMagnitudeRecursiveGaussian(ct_sitk, sigma_pix)
    grad_np = sitk.GetArrayFromImage(grad)

    # 4️⃣ Threshold gradient by percentile
    thresh_val = np.percentile(grad_np[grad_np>0], threshold_percentile)
    edge_mask = grad_np > thresh_val

    # 5️⃣ Plot
    plt.figure(figsize=(8, 8))
    #plt.imshow(ct_np[slice_idx, :, :], cmap='gray', vmin=hu_lower, vmax=3000)
    plt.imshow(ct_np[slice_idx, :, :], cmap='gray', vmin=-500, vmax=1500)

    plt.contour(mask[slice_idx, :, :], colors='r', linewidths=1, levels=[0.5])
    plt.contour(edge_mask[slice_idx, :, :], colors='cyan', linewidths=0.5, levels=[0.5])
    plt.title(f"Slice {slice_idx} — red: segmentation | cyan: edges")
    plt.axis('off')
    plt.show()

# --- Interactive sliders
interact(
    show_edges,
    slice_idx=IntSlider(min=0, max=ct_np_orig.shape[0]-1, step=1, value=ct_np_orig.shape[0]//2),
    sigma_mm=FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description="Gaussian σ (mm)"),
    hu_lower=IntSlider(min=-500, max=500, step=10, value=150, description="HU lower bound"),
    threshold_percentile=FloatSlider(min=50, max=99, step=1, value=85, description="Edge percentile")
)


interactive(children=(IntSlider(value=75, description='slice_idx', max=149), FloatSlider(value=1.0, descriptio…

<function __main__.show_edges(slice_idx, sigma_mm, hu_lower, threshold_percentile)>